In [22]:
!apt-get install ghostscript
!pip install camelot-py PyPDF2 pandas Flask flask-cors statsmodels matplotlib scikit-learn tensorflow
!pip install camelot-py tabula-py ghostscript openpyxl
!pip install pdfplumber plotly ipywidgets


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.11).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [23]:

import re
import camelot
import PyPDF2
import pandas as pd

desired_years = {"2020", "2021", "2022", "2023", "2024"}

#find a year token in a header cell
def extract_year_from_cell(cell):
    tokens = cell.split()         #split text into words
    for token in tokens:          #check each word
        if token in desired_years:  #if it matches one of the years
            return token
    return None

#locate first page containing all keywords
def find_first_page_with_all_keywords(pdf_path, keywords):
    with open(pdf_path, 'rb') as f:             #open PDF file
        reader = PyPDF2.PdfReader(f)            #create reader
        total_pages = len(reader.pages)         #count pages
        for i in range(total_pages):            #for each page index
            text = reader.pages[i].extract_text() or ""  #extract text
            if all(kw.lower() in text.lower() for kw in keywords):  #check keywords
                return i + 1                   #return page number (1‑based)
    return None

# --- Helper: pull tables from a page range ---
def extract_tables_from_range(pdf_path, start_page, end_page, flavor="stream"):
    page_range = f"{start_page}-{end_page}"      # e.g. "12-18"
    tables = camelot.read_pdf(pdf_path, pages=page_range, flavor=flavor)  # Read tables
    return tables                                # Return list of tables

#decide which column holds metric names
def determine_metric_column(header):
    if header and header[0].strip() == "Note":  #if first header is "Note"
        return 1                                 #metrics start in column 1
    return 0                                     #otherwise column 0

#find which row has the year headers
def find_header_row(df, max_rows=3):
    for row_idx in range(min(max_rows, len(df))):  #check up to max_rows
        row = [str(cell).strip() for cell in df.iloc[row_idx].tolist()]  #get row text
        for cell in row:                              #for each cell
            tokens = cell.split()                     #split into words
            for token in tokens:                      #for each token
                if token in desired_years:            #if it’s a year
                    return row_idx                     #return that row index
    return 0                                         #default to row 0

#extract metric values for all years
def extract_values_for_all_years(df, metric, header_row_index):
    header = [str(cell).strip() for cell in df.iloc[header_row_index].tolist()]  #clean header
    data = df.iloc[header_row_index + 1:].reset_index(drop=True)                #data rows
    metric_col_idx = determine_metric_column(header)                             #where metric names are

    #identify candidate columns (stop at "COMPANY" bcuz we want Group values only)
    start_candidate = metric_col_idx + 1
    end_candidate = len(header)
    for idx in range(start_candidate, len(header)):
        if header[idx].lower() == "company":
            end_candidate = idx
            break
    candidate_cols = list(range(start_candidate, end_candidate))  #columns to scan

    #if columns are duplicated (even count), only take first half
    if candidate_cols and len(candidate_cols) % 2 == 0:
        candidate_cols = candidate_cols[:len(candidate_cols) // 2]

    result = {}  #map year→raw value
    for col_idx in candidate_cols:
        col_header = header[col_idx]             #header text for this column
        found_year = extract_year_from_cell(col_header)  #extract year if present
        if not found_year or found_year in result:
            continue                             #skip if no year or already got it
        #find the row where the metric matches exactly
        for i in range(len(data)):
            if data.iloc[i, metric_col_idx].strip() == metric:
                result[found_year] = data.iloc[i, col_idx].strip()  #store raw string
                break
    return result

#process income statement section for one PDF
def process_income_statement_for_pdf(pdf_path, metrics, start_page, end_page):
    tables = extract_tables_from_range(pdf_path, start_page, end_page, flavor="stream")  #try stream mode
    if not tables or len(tables) == 0:                                                  #if no tables found
        tables = extract_tables_from_range(pdf_path, start_page, end_page, flavor="lattice")  #try lattice
        if not tables or len(tables) == 0:
            return {}

    results = {}  #year → {metric: raw_value}
    for table in tables:
        df = table.df                              #get DataFrame
        if df.shape[0] < 2:                        #skip small tables
            continue
        header_row_index = find_header_row(df, max_rows=3)  #locate header row
        for metric in metrics:                                 #for each metric name
            metric_values = extract_values_for_all_years(df, metric, header_row_index)
            for year, raw_val in metric_values.items():       #add to results if valid
                if year in desired_years:
                    results.setdefault(year, {})[metric] = raw_val
    return results

#convert raw strings into floats or None
def clean_number(val):
    if val is None or val.strip() == "":  #missing or empty
        return None
    if val.strip() == "-":               #dash means missing
        return None
    try:
        cleaned = (val.replace(',', '')   #remove commas
                      .replace('$', '')   #remove dollar signs
                      .replace('(', '-')  #parenthesis → minus
                      .replace(')', '')   #remove closing parenthesis
                      .strip())
        return float(cleaned)             #parse float
    except Exception:
        return None                       #parsing failed

#loop over PDFs, extract and save results
def main():
    pdf_files = [
        {"path": "/content/2020.pdf"},
        {"path": "/content/2021.pdf"},
        {"path": "/content/2022.pdf"},
        {"path": "/content/2023.pdf"}
    ]

    composite_keywords = [                        #keywords to find the income section
        "Continuing operations",
        "Revenue from contracts with customers",
        "Revenue from insurance contracts",
        "Total revenue"
    ]

    metrics = [                                   #line items to extract
        "Total revenue",
        "Cost of sales",
        "Gross profit",
        "Selling and distribution expenses",
        "Administrative expenses",
        "Other operating expenses"
    ]

    global_results = {}  #year → cleaned metric dict

    for pdf in pdf_files:                       #process each file
        pdf_path = pdf["path"]                  #get file path
        #find the first page containing the keywords
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            total_pages = len(reader.pages)
            first_page = None
            for i in range(total_pages):
                text = reader.pages[i].extract_text() or ""
                if all(kw.lower() in text.lower() for kw in composite_keywords):
                    first_page = i + 1
                    break
        #default or dynamic page range
        if first_page is None:
            start_page, end_page = 220, 260
        else:
            start_page = first_page
            end_page = first_page + 5

        #extract raw values
        pdf_results = process_income_statement_for_pdf(pdf_path, metrics, start_page, end_page)
        #clean and store
        for year, data in pdf_results.items():
            cleaned_data = {m: clean_number(data.get(m)) for m in metrics}
            global_results[year] = cleaned_data

    #build DataFrame
    final_list = []
    for year in sorted(global_results.keys(), key=lambda x: int(x) if x.isdigit() else 0):
        row = {"Year": year}
        row.update(global_results[year])
        final_list.append(row)
    df_results = pd.DataFrame(final_list).sort_values(by="Year")

    #save and display
    df_results.to_csv("group_consolidated_metrics_latest.csv", index=False)
    print("Final extracted metrics (latest values):")
    print(df_results)
    return df_results

if __name__ == "__main__":
    df_final = main()


/usr/local/lib/python3.11/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


Final extracted metrics (latest values):
   Year  Total revenue  Cost of sales  Gross profit  \
0  2020    138955752.0   -112874121.0    26081631.0   
1  2021    127675650.0   -108747058.0    18928592.0   
2  2022    218074746.0   -180430008.0    37644738.0   
3  2023    276639907.0   -227534456.0    49105451.0   
4  2024    280772604.0   -226490684.0    54281920.0   

   Selling and distribution expenses  Administrative expenses  \
0                         -5518526.0              -13143112.0   
1                         -4761037.0              -12927716.0   
2                         -5732694.0              -14762643.0   
3                         -8266060.0              -21583579.0   
4                        -10062663.0              -25172004.0   

   Other operating expenses  
0                -2872908.0  
1                -1314409.0  
2               -12456008.0  
3                -9825773.0  
4                -8188312.0  


In [24]:
import re
import camelot
import pandas as pd

desired_years = {"2020", "2021", "2022", "2023", "2024"}
file_path = "/content/2023.pdf"
target_page = "369"                #page number where the table is

#Row indices in the table for headers and specific metrics
HEADER_ROW_INDEX = 1            #The row that contains year tokens
BASIC_EPS_ROW_INDEX = 40        #The row that contains Basic EPS values
NET_PROFIT_ROW_INDEX = 10       #The row that contains "Profit after tax"
NET_ASSETS_ROW_INDEX = 42       #the row that contains Net assets per share

#clean a numeric string into float or None
def clean_number(val):
    """Remove commas, currency symbols, parentheses → negatives, then convert to float."""
    if not val or val.strip() in {"", "-"}:  #If empty or just a dash
        return None
    try:
        #Remove formatting characters and convert to float
        return float(
            val
            .replace(',', '')
            .replace('$', '')
            .replace('(', '-')
            .replace(')', '')
            .strip()
        )
    except Exception:
        return None

#extract a 4‑digit year from a token ---
def extract_year(token):
    """
    Find a substring matching 20XX in the token.
    Returns the year if it's in our desired_years set.
    """
    m = re.search(r"(20\d{2})", token)  #look for 20xx like 2020 2021 2022 etc
    if m:
        yr = m.group(1)                  #extract the year
        if yr in desired_years:         #check if it’s one needed
            return yr
    return None

#read tables from a given page range ---
def extract_tables_from_range(pdf_path, start_page, end_page, flavor="stream"):
    """Use Camelot to pull tables from pages start_page to end_page."""
    page_range = f"{start_page}-{end_page}"
    tables = camelot.read_pdf(pdf_path, pages=page_range, flavor=flavor)
    return tables

#pull year→value mapping for one metric row in a subtable
def extract_metric_from_subtable(df_sub, header_row_index, metric_row_index):
    """
    Given a small DataFrame chunk:
      - Read the header row for year tokens.
      - Read the metric row for values.
      - Return a dict {year: raw_value}.
    """
    header = [str(x).strip() for x in df_sub.iloc[header_row_index].tolist()]  #year labels
    metric_row = [str(x).strip() for x in df_sub.iloc[metric_row_index].tolist()]  #metric values
    mapping = {}
    for idx, token in enumerate(header):          #for each column index
        yr = extract_year(token)                  #try to parse a year
        if yr and idx < len(metric_row):          #if valid year and value exists
            mapping[yr] = metric_row[idx]         #map year → raw string
    return mapping

def main():
    #read the full table from the target page
    tables = extract_tables_from_range(file_path, target_page, target_page, flavor="stream")
    if not tables:
        tables = extract_tables_from_range(file_path, target_page, target_page, flavor="lattice")
    if not tables:
        print("No table found on page", target_page)
        return None

    df_full = tables[0].df  #use the first extracted table

    #split the table into left and right parts
    if df_full.shape[1] < 4:  #ensure enough columns to split
        print("Table too narrow for left/right split.")
        return None
    df_left = df_full.iloc[:, 0:3]   #first 3 columns (later years)
    df_right = df_full.iloc[:, 3:]   #remaining columns (earlier years)

    #extract raw mappings for each metric from both halves
    eps_left = extract_metric_from_subtable(df_left, HEADER_ROW_INDEX, BASIC_EPS_ROW_INDEX)
    np_left  = extract_metric_from_subtable(df_left, HEADER_ROW_INDEX, NET_PROFIT_ROW_INDEX)
    na_left  = extract_metric_from_subtable(df_left, HEADER_ROW_INDEX, NET_ASSETS_ROW_INDEX)

    eps_right = extract_metric_from_subtable(df_right, HEADER_ROW_INDEX, BASIC_EPS_ROW_INDEX)
    np_right  = extract_metric_from_subtable(df_right, HEADER_ROW_INDEX, NET_PROFIT_ROW_INDEX)
    na_right  = extract_metric_from_subtable(df_right, HEADER_ROW_INDEX, NET_ASSETS_ROW_INDEX)

    #merge left/right into unified dicts
    basic_eps_data = {**eps_left, **eps_right}
    net_profit_data = {**np_left, **np_right}
    net_assets_data = {**na_left, **na_right}

    #clean all numeric strings to floats
    basic_eps_clean = {yr: clean_number(v) for yr, v in basic_eps_data.items()}
    net_profit_clean = {yr: clean_number(v) for yr, v in net_profit_data.items()}
    net_assets_clean = {yr: clean_number(v) for yr, v in net_assets_data.items()}

    #compute share count (Net Profit ÷ Basic EPS)
    share_count = {}
    for yr in desired_years:
        eps_val = basic_eps_clean.get(yr)
        np_val = net_profit_clean.get(yr)
        if eps_val and np_val and eps_val != 0:
            share_count[yr] = np_val / eps_val

    #build final DataFrame and save
    years = sorted(desired_years, key=lambda x: int(x))  #sort chronologically
    rows = []
    for yr in years:
        rows.append({
            "Year": yr,
            "Basic_EPS": basic_eps_clean.get(yr),
            "Net_Profit": net_profit_clean.get(yr),
            "Net_Assets_Per_Share": net_assets_clean.get(yr),
            "Share_Count_Millions": share_count.get(yr)
        })
    df_metrics = pd.DataFrame(rows)
    df_metrics.to_csv("extracted_metrics.csv", index=False)
    print("\nFinal Extracted Metrics:")
    print(df_metrics)
    return df_metrics

if __name__ == "__main__":
    df_result = main()



Final Extracted Metrics:
   Year  Basic_EPS  Net_Profit  Net_Assets_Per_Share  Share_Count_Millions
0  2020       7.14      9741.0                 144.7           1364.285714
1  2021       3.62      3951.0                 150.9           1091.436464
2  2022      15.13     20443.0                 207.7           1351.156642
3  2023      13.12     18896.0                 227.5           1440.243902
4  2024       8.06     12128.0                 238.8           1504.714640


In [25]:
import re
import os
import camelot
import pandas as pd
import PyPDF2

pdf_files = ["2020.pdf", "2021.pdf", "2022.pdf", "2023.pdf"]

desired_years = {"2020", "2021", "2022", "2023", "2024"}

#clean numeric strings into float or None
def clean_number(val):
    """
    Remove commas, percent signs, currency symbols, parentheses → negatives, then convert to float.
    """
    if not val or val.strip() in {"", "-"}:  #skip empty or dash-only strings
        return None
    try:
        #strip formatting characters and parse float
        return float(
            val
            .replace(",", "")
            .replace("%", "")
            .replace("$", "")
            .replace("(", "-")
            .replace(")", "")
            .strip()
        )
    except Exception:
        return None

#extract a 4-digit year from text token ---
def extract_year(token):
    """
    Find a substring matching 20XX in the token and return it if it's desired.
    """
    m = re.search(r"(20\d{2})", token)  #look for '20' + two digits
    if m:
        yr = m.group(1)                  #extract the year
        if yr in desired_years:         #check if in the target set of year
            return yr
    return None

#find the first PDF page containing a phrase
def find_page_with_phrase(pdf_path, phrase):
    """
    Scan each page for phrase (case-insensitive). Return 1-based page number or None.
    """
    with open(pdf_path, 'rb') as f:               #open PDF file
        reader = PyPDF2.PdfReader(f)              #PDF reader
        total_pages = len(reader.pages)           #count pages
        for i in range(total_pages):              #iterate pages
            text = reader.pages[i].extract_text() or ""  #extract text or empty
            if phrase.lower() in text.lower():    #check for phrase
                return i + 1                      #return 1-based index
    return None

#extract Camelot tables from one page ---
def extract_tables_from_page(pdf_path, page_num):
    """
    Try 'stream' flavor first; if no tables, fall back to 'lattice'.
    Returns a list of table objects.
    """
    tables = camelot.read_pdf(pdf_path, pages=str(page_num), flavor="stream")
    if not tables or len(tables) == 0:
        tables = camelot.read_pdf(pdf_path, pages=str(page_num), flavor="lattice")
    return tables

#detect the Top Twenty Shareholders table
def is_shareholders_table(df):
    """
    Return True if one of the first three rows mentions 'top twenty shareholders'
    or if the table has exactly 5 columns.
    """
    for i in range(min(3, df.shape[0])):
        row_text = " ".join(str(x).lower() for x in df.iloc[i].tolist())
        if "top twenty shareholders" in row_text:
            return True
    if df.shape[1] == 5:
        return True
    return False

#read out each shareholder record from the table
def parse_shareholders_table(df, source_pdf):
    records = []
    if df.shape[0] < 3 or df.shape[1] != 5:
        return records

    subheader = [str(x).strip() for x in df.iloc[1].tolist()]  #year tokens row
    year1 = extract_year(subheader[1])          #extract first year
    year2 = extract_year(subheader[3])          #extract second year

    for i in range(2, df.shape[0]):             #iterate data rows
        row = [str(x).strip() for x in df.iloc[i].tolist()]
        if len(row) < 5:
            continue
        shareholder = row[0]                    #name in first column
        if not shareholder:
            continue
        perc1 = clean_number(row[2])            #ownership % for year1
        perc2 = clean_number(row[4])            #ownership % for year2
        if year1 and perc1 is not None:
            records.append({
                "Year": year1,
                "Shareholder": shareholder,
                "Ownership_Percentage": perc1,
                "Source_PDF": source_pdf
            })
        if year2 and perc2 is not None:
            records.append({
                "Year": year2,
                "Shareholder": shareholder,
                "Ownership_Percentage": perc2,
                "Source_PDF": source_pdf
            })
    return records

#normalize shareholder names for grouping
def normalize_name(name):
    """
    Lowercase, trim, and replace '&' with 'and'. Extend as needed.
    """
    name = name.lower().strip()                  #lowercase and trim
    name = name.replace("&", "and")
    return name

#find and parse the shareholders table in one PDF
def extract_shareholders_from_pdf(pdf_path):
    phrase = "top twenty shareholders"            #Text to look for
    page = find_page_with_phrase(pdf_path, phrase)
    if not page:                                  #if phrase not found
        print(f"'{phrase}' not found in {os.path.basename(pdf_path)}. Scanning all pages.")
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            total_pages = len(reader.pages)
        pages_to_try = range(1, total_pages + 1)  #try every page
    else:
        pages_to_try = [page]                     #only that page

    for p in pages_to_try:
        tables = extract_tables_from_page(pdf_path, p)
        if not tables:
            continue
        for table in tables:
            df = table.df
            if is_shareholders_table(df):
                recs = parse_shareholders_table(df, os.path.basename(pdf_path))
                if recs:
                    return recs
    return []

#combine all PDFs into one raw DataFrame
def combine_shareholders_data(pdf_files):
    all_records = []
    for pdf in pdf_files:
        recs = extract_shareholders_from_pdf(pdf)
        if recs:
            all_records.extend(recs)
    if all_records:
        return pd.DataFrame(all_records)
    else:
        return pd.DataFrame(columns=["Year", "Shareholder", "Ownership_Percentage", "Source_PDF"])

#consolidate duplicates by normalized name & year
def consolidate_duplicates(df):
    df["Normalized_Shareholder"] = df["Shareholder"].apply(normalize_name)
    df_grouped = df.groupby(["Normalized_Shareholder", "Year"], as_index=False) \
                   .agg({"Ownership_Percentage": "first"})
    df_grouped = df_grouped.rename(columns={"Normalized_Shareholder": "Shareholder"})
    return df_grouped

#pivot so each shareholder is a row, years are columns
def pivot_shareholders_data(df):
    if df.empty:
        return df
    df_pivot = df.pivot(index="Shareholder", columns="Year", values="Ownership_Percentage")
    sorted_years = sorted([col for col in df_pivot.columns if col.isdigit()], key=int)
    df_pivot = df_pivot.reindex(columns=sorted_years)
    df_pivot = df_pivot.rename(columns={yr: f"{yr} (%)" for yr in sorted_years}).reset_index()
    return df_pivot

def main():
    df_raw = combine_shareholders_data(pdf_files)    #extract raw records
    if df_raw.empty:
        return df_raw

    df_raw = df_raw[df_raw["Year"].isin(desired_years)]  #filter by the target years
    df_consolidated = consolidate_duplicates(df_raw)      #merge duplicates
    df_pivot = pivot_shareholders_data(df_consolidated)  #
    df_pivot.to_csv("top20_shareholders.csv", index=False)

    print("\nFinal Top 20 Shareholders Data:")
    print(df_pivot)
    return df_pivot

if __name__ == "__main__":
    final_df = main()



Final Top 20 Shareholders Data:
Year                                        Shareholder  2020 (%)  2021 (%)  \
0                      aberdeen standard asia focus plc      1.72      1.81   
1                                asian development bank       NaN       NaN   
2                                          chemanex plc      0.40      0.87   
3                                      cic holdings plc       NaN       NaN   
4     citigroup global markets limited agency tradin...     10.76     10.75   
5                          edgbaston asian equity trust      2.26      1.12   
6                             employee's provident fund      0.77      0.77   
7                            employees trust fund board      1.42      1.42   
8                              emrevival master fund lp       NaN       NaN   
9                               fidelity fund - pacific      1.45      1.35   
10                 hostplus pooled superannuation trust       NaN       NaN   
11                 

In [ ]:
df_extracted_intial = pd.read_csv("/content/group_consolidated_metrics_latest.csv")
df_extracted_intial

,Year,Total revenue,Cost of sales,Gross profit,Selling and distribution expenses,Administrative expenses,Other operating expenses
0,2020,138955752.0,-112874121.0,26081631.0,-5518526.0,-13143112.0,-2872908.0
1,2021,127675650.0,-108747058.0,18928592.0,-4761037.0,-12927716.0,-1314409.0
2,2022,218074746.0,-180430008.0,37644738.0,-5732694.0,-14762643.0,-12456008.0
3,2023,276639907.0,-227534456.0,49105451.0,-8266060.0,-21583579.0,-9825773.0
4,2024,280772604.0,-226490684.0,54281920.0,-10062663.0,-25172004.0,-8188312.0


In [ ]:
for col in df_extracted_intial.select_dtypes(include='number').columns:
    df_extracted_intial[col] = df_extracted_intial[col].abs()

df_extracted_intial

,Year,Total revenue,Cost of sales,Gross profit,Selling and distribution expenses,Administrative expenses,Other operating expenses
0,2020,138955752.0,112874121.0,26081631.0,5518526.0,13143112.0,2872908.0
1,2021,127675650.0,108747058.0,18928592.0,4761037.0,12927716.0,1314409.0
2,2022,218074746.0,180430008.0,37644738.0,5732694.0,14762643.0,12456008.0
3,2023,276639907.0,227534456.0,49105451.0,8266060.0,21583579.0,9825773.0
4,2024,280772604.0,226490684.0,54281920.0,10062663.0,25172004.0,8188312.0


In [ ]:
df_extracted_intial['Total operating expenses'] = df_extracted_intial['Selling and distribution expenses'] + df_extracted_intial['Administrative expenses'] + df_extracted_intial['Other operating expenses']

df_extracted_intial

,Year,Total revenue,Cost of sales,Gross profit,Selling and distribution expenses,Administrative expenses,Other operating expenses,Total operating expenses
0,2020,138955752.0,112874121.0,26081631.0,5518526.0,13143112.0,2872908.0,21534546.0
1,2021,127675650.0,108747058.0,18928592.0,4761037.0,12927716.0,1314409.0,19003162.0
2,2022,218074746.0,180430008.0,37644738.0,5732694.0,14762643.0,12456008.0,32951345.0
3,2023,276639907.0,227534456.0,49105451.0,8266060.0,21583579.0,9825773.0,39675412.0
4,2024,280772604.0,226490684.0,54281920.0,10062663.0,25172004.0,8188312.0,43422979.0


In [ ]:
df_extracted_intial['Gross profit margin'] = (df_extracted_intial['Gross profit'] / df_extracted_intial['Total revenue']) * 100
df_extracted_intial

,Year,Total revenue,Cost of sales,Gross profit,Selling and distribution expenses,Administrative expenses,Other operating expenses,Total operating expenses,Gross profit margin
0,2020,138955752.0,112874121.0,26081631.0,5518526.0,13143112.0,2872908.0,21534546.0,18.769738
1,2021,127675650.0,108747058.0,18928592.0,4761037.0,12927716.0,1314409.0,19003162.0,14.825530
2,2022,218074746.0,180430008.0,37644738.0,5732694.0,14762643.0,12456008.0,32951345.0,17.262310
3,2023,276639907.0,227534456.0,49105451.0,8266060.0,21583579.0,9825773.0,39675412.0,17.750675
4,2024,280772604.0,226490684.0,54281920.0,10062663.0,25172004.0,8188312.0,43422979.0,19.333054


In [ ]:
df_extracted_intial.to_csv("group_consolidated_data.csv", index=False)

In [ ]:
import pandas as pd

metrics_df = pd.read_csv('/content/extracted_metrics.csv')
group_df = pd.read_csv('/content/group_consolidated_data.csv')

print("Metrics columns:", metrics_df.columns)
print("Group columns:", group_df.columns)



Metrics columns: Index(['Year', 'Basic_EPS', 'Net_Profit', 'Net_Assets_Per_Share',
       'Share_Count_Millions'],
      dtype='object')
Group columns: Index(['Year', 'Total revenue', 'Cost of sales', 'Gross profit',
       'Selling and distribution expenses', 'Administrative expenses',
       'Other operating expenses', 'Total operating expenses',
       'Gross profit margin'],
      dtype='object')


In [ ]:
#merge the datasets based on the common column
merged_df = pd.merge(metrics_df, group_df, on="Year", how="outer")
merged_df

,Year,Basic_EPS,Net_Profit,Net_Assets_Per_Share,Share_Count_Millions,Total revenue,Cost of sales,Gross profit,Selling and distribution expenses,Administrative expenses,Other operating expenses,Total operating expenses,Gross profit margin
0,2020,7.14,9741.0,144.7,1364.285714,138955752.0,112874121.0,26081631.0,5518526.0,13143112.0,2872908.0,21534546.0,18.769738
1,2021,3.62,3951.0,150.9,1091.436464,127675650.0,108747058.0,18928592.0,4761037.0,12927716.0,1314409.0,19003162.0,14.825530
2,2022,15.13,20443.0,207.7,1351.156642,218074746.0,180430008.0,37644738.0,5732694.0,14762643.0,12456008.0,32951345.0,17.262310
3,2023,13.12,18896.0,227.5,1440.243902,276639907.0,227534456.0,49105451.0,8266060.0,21583579.0,9825773.0,39675412.0,17.750675
4,2024,8.06,12128.0,238.8,1504.714640,280772604.0,226490684.0,54281920.0,10062663.0,25172004.0,8188312.0,43422979.0,19.333054


In [ ]:
merged_df.to_csv("combined_data.csv", index=False)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/top20_shareholders.csv')
df = df.replace([np.nan, np.inf, -np.inf], 0)
df

,Shareholder,2020 (%),2021 (%),2022 (%),2023 (%),2024 (%)
0,aberdeen standard asia focus plc,1.72,1.81,2.03,2.41,2.34
1,asian development bank,0.00,0.00,4.70,4.70,4.34
2,chemanex plc,0.40,0.87,0.91,0.95,1.01
3,cic holdings plc,0.00,0.00,0.00,2.55,5.70
4,citigroup global markets limited agency tradin...,10.76,10.75,4.47,4.47,0.00
5,edgbaston asian equity trust,2.26,1.12,1.09,1.27,0.95
6,employee's provident fund,0.77,0.77,0.00,0.00,0.00
7,employees trust fund board,1.42,1.42,1.34,1.34,0.87
8,emrevival master fund lp,0.00,0.00,0.00,0.00,1.07
9,fidelity fund - pacific,1.45,1.35,1.29,1.10,0.00


In [ ]:
df['Shareholder'] = df['Shareholder'].str.title()
df

,Shareholder,2020 (%),2021 (%),2022 (%),2023 (%),2024 (%)
0,Aberdeen Standard Asia Focus Plc,1.72,1.81,2.03,2.41,2.34
1,Asian Development Bank,0.00,0.00,4.70,4.70,4.34
2,Chemanex Plc,0.40,0.87,0.91,0.95,1.01
3,Cic Holdings Plc,0.00,0.00,0.00,2.55,5.70
4,Citigroup Global Markets Limited Agency Tradin...,10.76,10.75,4.47,4.47,0.00
5,Edgbaston Asian Equity Trust,2.26,1.12,1.09,1.27,0.95
6,Employee'S Provident Fund,0.77,0.77,0.00,0.00,0.00
7,Employees Trust Fund Board,1.42,1.42,1.34,1.34,0.87
8,Emrevival Master Fund Lp,0.00,0.00,0.00,0.00,1.07
9,Fidelity Fund - Pacific,1.45,1.35,1.29,1.10,0.00


In [ ]:
df.to_csv("final_top20_shareholders.csv", index=False)